In [1]:
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt
import os
import glob

Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [3]:
from keras.models import model_from_json
model.load_weights('vgg_face_weights.h5')

In [4]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [5]:
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [6]:
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

In [7]:
epsilon = 0.25
epsilon1=50

def verifyFace(img1, img2):
    img1_representation = vgg_face_descriptor.predict(preprocess_image(img1))[0,:]
    img2_representation = vgg_face_descriptor.predict(preprocess_image(img2))[0,:]
    
    cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
    euclidean_distance = findEuclideanDistance(img1_representation, img2_representation)
    
    print("Cosine similarity: ",cosine_similarity)
    print("Euclidean distance: ",euclidean_distance)
    
    if(cosine_similarity < epsilon) and (euclidean_distance < epsilon1):
        print("verified... they are same person")
    else:
        print("unverified! they are not same person!")
    
    f = plt.figure()
    f.add_subplot(1,2, 1)
    plt.imshow(image.load_img(img1))
    plt.xticks([]); plt.yticks([])
    f.add_subplot(1,2, 2)
    plt.imshow(image.load_img(img2))
    plt.xticks([]); plt.yticks([])
    plt.show(block=True)
    print("-----------------------------------------")

## Test 1 Photo:Cosine similarity

In [51]:

def test(Image):
    a=1000
    j=''
    img1_representation = vgg_face_descriptor.predict(preprocess_image(Image))[0,:]
    for i in glob.glob("BDD/*"):
        img2_representation = vgg_face_descriptor.predict(preprocess_image(i))[0,:]
        cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
        if cosine_similarity<a:
            a=cosine_similarity
            j=i
    if Image[5:-8]==j[4:-8]:
        return 1
    else:
        return 0

In [52]:
n,k=0,0
for i in glob.glob("Test/*"):
    n=n+1
    k=k+test(i)
k*100/n

79.16666666666667

## Test 1 photo:euclidean distance

In [53]:
def test1(Image):
    a=1000
    j=''
    img1_representation = vgg_face_descriptor.predict(preprocess_image(Image))[0,:]
    for i in glob.glob("BDD/*"):
        img2_representation = vgg_face_descriptor.predict(preprocess_image(i))[0,:]
        euclidean_distance = findEuclideanDistance(img1_representation, img2_representation)
        if euclidean_distance<a:
            a=euclidean_distance
            j=i
    if Image[5:-8]==j[4:-8]:
        return 1
    else:
        return 0

In [54]:
n,k=0,0
for i in glob.glob("Test/*"):
    n=n+1
    k=k+test1(i)
k*100/n

75.0

## Test 2 Photos:Cosine similarity

In [55]:
def test(Image):
    a=1000
    j=''
    img1_representation = vgg_face_descriptor.predict(preprocess_image(Image))[0,:]
    for i in glob.glob("BDD/*"):
        img2_representation = vgg_face_descriptor.predict(preprocess_image(i))[0,:]
        cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
        if cosine_similarity<a:
            a=cosine_similarity
            j=i
    if Image[5:-8]==j[4:-8]:
        return 1
    else:
        return 0

In [56]:
n,k=0,0
for i in glob.glob("Test/*"):
    n=n+1
    k=k+test(i)
k*100/n

95.83333333333333

## Change Contrast

In [57]:
from PIL import Image


def change_contrast(img, level):

    img = Image.open("Test/Aaron_Peirsol_0002.jpg")
    img.load()

    factor = (259 * (level+255)) / (255 * (259-level))
    for x in range(img.size[0]):
        for y in range(img.size[1]):
            color = img.getpixel((x, y))
            new_color = tuple(int(factor * (c-128) + 128) for c in color)
            img.putpixel((x, y), new_color)

    return img

done


In [65]:
#for file in  glob.glob("Test/*"):
 #   result=change_contrast(file, 100)
  #  result.save('Contrast/Contrast_'+file[5:])

In [71]:
def testC(Image):
    a=1000
    j=''
    img1_representation = vgg_face_descriptor.predict(preprocess_image(Image))[0,:]
    for i in glob.glob("BDD/*"):
        img2_representation = vgg_face_descriptor.predict(preprocess_image(i))[0,:]
        cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
        if cosine_similarity<a:
            a=cosine_similarity
            j=i
    if Image[18:-8]==j[4:-8]:
        return 1
    else:
        return 0

In [72]:
n,k=0,0
for i in glob.glob("Contrast/*"):
    n=n+1
    k=k+testC(i)
k*100/n

4.166666666666667

## Rotation

In [77]:
from PIL import Image
for file in  glob.glob("Test/*"):
    im = Image.open(file)
    im1=im.rotate(45)
    im1.save("Rotation/Rot_"+file[5:])

In [86]:
def testR(Image):
    a=1000
    j=''
    img1_representation = vgg_face_descriptor.predict(preprocess_image(Image))[0,:]
    for i in glob.glob("BDD/*"):
        img2_representation = vgg_face_descriptor.predict(preprocess_image(i))[0,:]
        cosine_similarity = findCosineSimilarity(img1_representation, img2_representation)
        if cosine_similarity<a:
            a=cosine_similarity
            j=i
    if Image[13:-8]==j[4:-8]:
        return 1
    else:
        return 0

In [87]:
n,k=0,0
for i in glob.glob("Rotation/*"):
    n=n+1
    k=k+testR(i)
k*100/n

29.166666666666668